In [1]:
%cd /home/dvc-2-iris-demo

/home/dvc-2-iris-demo


In [12]:
# Remove old dvc files to avoid dependencies duplication error

# !rm $(find . -name "*?.dvc")

# Overview main config

In [3]:
import yaml

config = yaml.load(open('config/pipeline_config.yml'), Loader=yaml.FullLoader)
config

{'base': {'project': '7labs/dvc-2-iris-demo',
  'name': 'iris',
  'tags': ['solution-0-prototype', 'dev'],
  'model': {'model_name': 'model.joblib', 'models_folder': 'models'},
  'experiments': {'experiments_folder': 'experiments'},
  'random_state': 42},
 'split_train_test': {'folder': 'experiments',
  'train_csv': 'data/processed/train_iris.csv',
  'test_csv': 'data/processed/test_iris.csv',
  'test_size': 0.2},
 'featurize': {'dataset_csv': 'data/raw/iris.csv',
  'featured_dataset_csv': 'data/interim/featured_iris.csv',
  'features_columns_range': ['sepal_length', 'petal_length_to_petal_width'],
  'target_column': 'species'},
 'train': {'cv': 5,
  'estimator_name': 'logreg',
  'estimators': {'logreg': {'param_grid': {'C': [0.001, 0.01],
     'max_iter': [100],
     'solver': ['lbfgs'],
     'multi_class': ['multinomial']}},
   'knn': {'param_grid': {'n_neighbors': [5, 15], 'p': [1, 2]}},
   'svm': {'param_grid': {'C': [0.1, 1.0],
     'kernel': ['rbf', 'linear'],
     'gamma': ['sca

# Create and run pipelines

## 1. Setup / config

In [3]:
!dvc run -f stage_prepare_configs.dvc \
    -d src/pipelines/prepare_configs.py \
    -d config/pipeline_config.yml \
    -o experiments/split_train_test_config.yml \
    -o experiments/featurize_config.yml \
    -o experiments/train_config.yml \
    -o experiments/evaluate_config.yml \
    python src/pipelines/prepare_configs.py \
        --config=config/pipeline_config.yml

Running command:
	python src/pipelines/prepare_configs.py --config=config/pipeline_config.yml
Save config: experiments/base_config.yml
Save config: experiments/split_train_test_config.yml
Save config: experiments/featurize_config.yml
Save config: experiments/train_config.yml
Save config: experiments/evaluate_config.yml
Saving 'experiments/split_train_test_config.yml' to '.dvc/cache/7f/cf7d4c56cd36d485c9776c8dfbafa7'.
Saving 'experiments/featurize_config.yml' to '.dvc/cache/3d/9c113b409731fba887d5a9cd43b486'.
Saving 'experiments/train_config.yml' to '.dvc/cache/09/62f3873bcc28078851867c92d22fc3'.
Saving 'experiments/evaluate_config.yml' to '.dvc/cache/91/c85b95fcb52f7b1957c02d11143a2d'.
Saving information to 'stage_prepare_configs.dvc'.

To track the changes with git run:

	git add stage_prepare_configs.dvc


## 2. Featurization

In [4]:
!dvc run -f stage_featurize.dvc \
    -d src/pipelines/featurize.py \
    -d experiments/featurize_config.yml \
    -d data/raw/iris.csv \
    -o data/interim/featured_iris.csv \
    python src/pipelines/featurize.py \
        --config=experiments/featurize_config.yml

Running command:
	python src/pipelines/featurize.py --config=experiments/featurize_config.yml
Saving information to 'stage_featurize.dvc'.

To track the changes with git run:

	git add stage_featurize.dvc


## 3. Split dataset into train/test

In [5]:
!dvc run -f stage_split_train_test.dvc \
    -d src/pipelines/split_train_test.py \
    -d experiments/split_train_test_config.yml \
    -d data/interim/featured_iris.csv \
    -o data/processed/train_iris.csv \
    -o data/processed/test_iris.csv \
    python src/pipelines/split_train_test.py \
        --config=experiments/split_train_test_config.yml \
        --base_config=config/pipeline_config.yml

Running command:
	python src/pipelines/split_train_test.py --config=experiments/split_train_test_config.yml --base_config=config/pipeline_config.yml
Saving 'data/processed/train_iris.csv' to '.dvc/cache/36/6af2683b508936c8c83e963e88cd40'.
Saving 'data/processed/test_iris.csv' to '.dvc/cache/76/51c34292c7d846debd07ed83897606'.
Saving information to 'stage_split_train_test.dvc'.

To track the changes with git run:

	git add stage_split_train_test.dvc


## 4. Train

In [6]:
!dvc run -f stage_train.dvc \
    -d src/pipelines/train.py \
    -d experiments/train_config.yml \
    -d data/processed/train_iris.csv \
    -o models/model.joblib \
    python src/pipelines/train.py \
        --config=experiments/train_config.yml \
        --base_config=config/pipeline_config.yml

Running command:
	python src/pipelines/train.py --config=experiments/train_config.yml --base_config=config/pipeline_config.yml
Fitting 5 folds for each of 2 candidates, totalling 10 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
0.8529086479396076
Saving 'models/model.joblib' to '.dvc/cache/d8/6199364bf884ace51244a2c5bdaa14'.
Saving information to 'stage_train.dvc'.

To track the changes with git run:

	git add stage_train.dvc


## 5. Evaluate 

In [7]:
!dvc run -f stage_evaluate.dvc \
    -d src/pipelines/evaluate.py \
    -d experiments/evaluate_config.yml \
    -d models/model.joblib \
    -m experiments/eval.txt \
    python src/pipelines/evaluate.py \
        --config=experiments/evaluate_config.yml \
        --base_config=config/pipeline_config.yml

Running command:
	python src/pipelines/evaluate.py --config=experiments/evaluate_config.yml --base_config=config/pipeline_config.yml
{'f1_score': 0.9305555555555555, 'confusion_matrix': [[10, 0, 0], [0, 7, 0], [0, 2, 11]]}
Saving 'experiments/eval.txt' to '.dvc/cache/d9/e6179f82ccc574b27763a38177f3d7'.
Saving information to 'stage_evaluate.dvc'.

To track the changes with git run:

	git add stage_evaluate.dvc


# Reproduce pipeline

### Evaluate (final stage)

In [9]:
# Pipeline is up to date. Nothing to reproduce.

!dvc repro stage_evaluate.dvc

Stage 'stage_prepare_configs.dvc' didn't change.
Stage 'stage_featurize.dvc' didn't change.
Stage 'stage_split_train_test.dvc' didn't change.
Stage 'stage_train.dvc' didn't change.
Stage 'stage_evaluate.dvc' didn't change.
Pipeline is up to date. Nothing to reproduce.


In [13]:
# -f force reproducing pipeline

!dvc repro stage_evaluate.dvc -f

Stage 'stage_prepare_configs.dvc' didn't change.
Reproducing 'stage_prepare_configs.dvc'
Running command:
	python src/pipelines/prepare_configs.py --config=config/pipeline_config.yml
Save config: experiments/base_config.yml
Save config: experiments/split_train_test_config.yml
Save config: experiments/featurize_config.yml
Save config: experiments/train_config.yml
Save config: experiments/evaluate_config.yml
Output 'experiments/split_train_test_config.yml' didn't change. Skipping saving.
Output 'experiments/featurize_config.yml' didn't change. Skipping saving.
Output 'experiments/train_config.yml' didn't change. Skipping saving.
Output 'experiments/evaluate_config.yml' didn't change. Skipping saving.
Saving information to 'stage_prepare_configs.dvc'.
Stage 'stage_featurize.dvc' didn't change.
Reproducing 'stage_featurize.dvc'
Running command:
	python src/pipelines/featurize.py --config=experiments/featurize_config.yml
Output 'data/interim/featured_iris.csv' didn't change. Skipping saving